In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D

import numpy as np

from datetime import datetime
import os
import sys

In [6]:
# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

# Data

In [3]:
# sys.path.insert(0, '/gpfs/data/paulab/nj594/ecg_ptbxl_benchmarking/code')
# from utils import utils

# #1. Data Parameters
# datafolder = '/gpfs/data/paulab/nj594/ecg_ptbxl_benchmarking/data/ptbxl/'
# data_dir = os.path.join(os.getcwd(), 'data')
# if not os.path.isdir(data_dir):
#     os.makedirs(data_dir)
# task = 'subdiagnostic'
# sampling_frequency=100 
# min_samples=0 
# train_fold=8 
# val_fold=9 
# test_fold=10 
# folds_type='strat'

# #2. Load PTB-XL data
# data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)

# #3. Preprocess label data
# labels = utils.compute_label_aggregations(raw_labels, datafolder, task)

# #4. Select relevant data and convert to one-hot
# data, labels, Y, _ = utils.select_data(data, labels, task, min_samples, data_dir+'/')
# input_shape = data[0].shape

# # 10th fold for testing (9th for now)
# X_test = data[labels.strat_fold == test_fold]
# y_test = Y[labels.strat_fold == test_fold]
# # 9th fold for validation (8th for now)
# X_val = data[labels.strat_fold == val_fold]
# y_val = Y[labels.strat_fold == val_fold]
# # rest for training
# X_train = data[labels.strat_fold <= train_fold]
# y_train = Y[labels.strat_fold <= train_fold]

# #5. Preprocess signal data
# X_train, X_val, X_test = utils.preprocess_signals(X_train, X_val, X_test, data_dir+'/')
# n_classes = y_train.shape[1]

# #6. Extract Label
# def get_label(X, y, label_inds):
#     #Assign Label 
#     y = ((y[:, label_inds[0]] == 1) |  (y[:, label_inds[1]] == 1))
    
#     # Select Lead V1
#     X = X[:, :, 6][:,:,np.newaxis] # Select Lead V1
    
    
#     return X, y

# X_train, y_train = get_label(X_train, y_train, [2,5])
# X_val, y_val = get_label(X_val, y_val, [2,5])
# X_test, y_test = get_label(X_test, y_test, [2,5])

# # convert class vectors to binary class matrices
# y_train = tf.keras.utils.to_categorical(y_train, 2)
# y_val = tf.keras.utils.to_categorical(y_val, 2)
# y_test = tf.keras.utils.to_categorical(y_test, 2)

# #7. Save Data
# X_train.dump(os.path.join(data_dir, 'X_train.npy'))
# X_val.dump(os.path.join(data_dir, 'X_val.npy'))
# X_test.dump(os.path.join(data_dir, 'X_test.npy'))

# y_train.dump(os.path.join(data_dir, 'y_train.npy'))
# y_val.dump(os.path.join(data_dir, 'y_val.npy'))
# y_test.dump(os.path.join(data_dir, 'y_test.npy'))

In [7]:
data_dir = os.path.join(os.getcwd(), 'data')

X_train = np.load(os.path.join(data_dir, 'X_train.npy'), allow_pickle=True)
X_val = np.load(os.path.join(data_dir, 'X_val.npy'), allow_pickle=True)
X_test = np.load(os.path.join(data_dir, 'X_test.npy'), allow_pickle=True)

y_train = np.load(os.path.join(data_dir, 'y_train.npy'), allow_pickle=True)
y_val = np.load(os.path.join(data_dir, 'y_val.npy'), allow_pickle=True)
y_test = np.load(os.path.join(data_dir, 'y_test.npy'), allow_pickle=True)

# Train Model to Be Explained

### Parameters

In [7]:
EPOCHS = 100
LR = 1e-3
BATCH_SIZE = 32

params = {
    #NN Hyperparameters
    "input_shape": [1000, 1],
    "num_categories": 2,
    "conv_subsample_lengths": [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2],
    "conv_filter_length": 8,
    "conv_num_filters_start": 32,
    "conv_init": "he_normal",
    "conv_activation": "relu",
    "conv_dropout": 0.2,
    "conv_num_skip": 2,
    "conv_increase_channels_at": 4,
    "compile": False,
    "is_regular_conv": False,
    "is_by_time": False,
    "is_by_lead": False,
    "ecg_out_size": 64,
    "nn_layer_sizes" : None,
    "is_multiply_layer": False, 
}

### Model

In [8]:
#Stanford Model
sys.path.insert(0, '/gpfs/data/paulab/nj594/ecg/models/stanford')
import network

model_input = Input(shape=(1000,1))

cnn = network.build_network(**params) 
cnn = Model(cnn.inputs, cnn.layers[-4].output)

net = cnn(model_input)
net = GlobalAveragePooling1D()(net)
out = Dense(2, activation='softmax')(net)

model = Model(model_input, out)
model.summary()

# Metrics
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
]

# Model Checkpointing
save_dir = 'model'
model_dir = os.path.join(os.getcwd(), save_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
model_weights_path = os.path.join(model_dir, 'model_weights.h5')
checkpoint = ModelCheckpoint(model_weights_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

# LR Schedule
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, 
                             verbose=1, mode='min', cooldown=1, min_lr=LR/100)

# Early Stopping 
earlyStop = EarlyStopping(monitor="val_loss", mode="min", patience=10) 

# Compile Model
CALLBACKS = [checkpoint, earlyStop, reduceLR]
OPTIMIZER = tf.keras.optimizers.Adam(LR)

model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

# Train Model
model.fit(x = X_train,
          y = y_train,
          batch_size = BATCH_SIZE,
          epochs = EPOCHS,
          validation_data = (X_val, y_val),
          callbacks = CALLBACKS)

# Get Checkpointed Model
print(model_weights_path)
model.load_weights(model_weights_path)
model.trainable = False

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 1)]         0         
_________________________________________________________________
functional_1 (Functional)    (None, 4, 256)            5246112   
_________________________________________________________________
global_average_pooling1d (Gl (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 5,246,626
Trainable params: 5,238,882
Non-trainable params: 7,744
_________________________________________________________________
Epoch 1/100
535/535 [==============================] - ETA: 0s - loss: 0.1285 - auroc: 0.9880 - auprc: 0.9870 - accuracy: 0.9582
Epoch 00001: val_loss improved from inf to 0.10111, saving model to /gpfs/data/paulab/nj594/ecg_explain/exper

### Save Predictions

In [3]:
params = {
    #NN Hyperparameters
    "input_shape": [1000, 1],
    "num_categories": 2,
    "conv_subsample_lengths": [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2],
    "conv_filter_length": 8,
    "conv_num_filters_start": 32,
    "conv_init": "he_normal",
    "conv_activation": "relu",
    "conv_dropout": 0.2,
    "conv_num_skip": 2,
    "conv_increase_channels_at": 4,
    "compile": False,
    "is_regular_conv": False,
    "is_by_time": False,
    "is_by_lead": False,
    "ecg_out_size": 64,
    "nn_layer_sizes" : None,
    "is_multiply_layer": False, 
}

#Stanford Model
sys.path.insert(0, '/scratch/nj594/ecg/models/stanford')
import network

model_input = Input(shape=(1000,1))

cnn = network.build_network(**params) 
cnn = Model(cnn.inputs, cnn.layers[-4].output)

net = cnn(model_input)
net = GlobalAveragePooling1D()(net)
out = Dense(2, activation='softmax')(net)

model = Model(model_input, out)
model.summary()

# Model Checkpointing
save_dir = 'model'
model_dir = os.path.join(os.getcwd(), save_dir)
model_weights_path = os.path.join(model_dir, 'model_weights.h5')

# Get Checkpointed Model
print(model_weights_path)
model.load_weights(model_weights_path)
model.trainable = False

2022-05-11 17:33:14.666732: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-11 17:33:14.667136: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 32252 MB memory:  -> device: 0, name: Vega 20, pci bus id: 0000:c8:00.0


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 1)]         0         
_________________________________________________________________
model (Functional)           (None, 4, 256)            5246112   
_________________________________________________________________
global_average_pooling1d (Gl (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 5,246,626
Trainable params: 5,238,882
Non-trainable params: 7,744
_________________________________________________________________
/vast/nj594/xai/ecg/experiments/model/model_weights.h5


In [8]:
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
]

OPTIMIZER = tf.keras.optimizers.Adam(1e-3)

model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)
model.evaluate(x=X_test, y=y_test)

2022-05-11 17:33:35.330484: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-11 17:33:35.348751: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-11 17:33:35.352499: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-11 17:33:35.355223: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-11 17:33:36.220582: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


68/68 [==============================] - 8s 17ms/step - loss: 0.0651 - auroc: 0.9967 - auprc: 0.9964 - accuracy: 0.9713


[0.06508663296699524,
 0.9967055320739746,
 0.9963754415512085,
 0.9713361263275146]

In [9]:
# Load Evaluator Model

eval_dir = os.path.join('evaluation', 'evaluator-data')
evaluator_model = tf.keras.models.load_model(os.path.join(eval_dir, 'surrogate.h5'))

OPTIMIZER = tf.keras.optimizers.Adam(1e-3)
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

model.evaluate(x=X_test, y=y_test)

 8/68 [==>...........................] - ETA: 1s - loss: 0.0308 - auroc: 0.9995 - auprc: 0.9995 - accuracy: 0.9844

2022-05-11 17:34:32.229454: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-11 17:34:32.232981: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-11 17:34:32.235667: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


68/68 [==============================] - 1s 17ms/step - loss: 0.0651 - auroc: 0.9967 - auprc: 0.9964 - accuracy: 0.9713


[0.06508663296699524,
 0.9967055320739746,
 0.9963754415512085,
 0.9713361263275146]

In [ ]:
#predictions
predictions = model.predict(X_test)
predictions.dump(os.path.join(data_dir, 'predictions.npy'))

predictions_val = model.predict(X_val)
predictions_val.dump(os.path.join(data_dir, 'predictions_val.npy'))

In [6]:
predictions_train = model.predict(X_train)
predictions_train.dump(os.path.join(data_dir, 'predictions_train.npy'))

2021-10-13 17:40:47.549434: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-13 17:40:47.568654: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2021-10-13 17:40:47.572543: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2021-10-13 17:40:47.574848: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2021-10-13 17:40:48.179889: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#52
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#52
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#52
MIOpen

MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#52
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#52
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#52
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#52
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#52
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#52
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#52
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key n